# 1. Google Cloud: API Gateway の使い方

[Cloud Run](https://cloud.google.com/run?hl=ja) をバックエンドに、API Gateway をセットアップしてみましょう。

## 1.1. このセクションのゴール

prod-gateway というゲートウェイと 2 つの構成ををもつ、my-apis と名付けた API を定義します。  
バックエンドには 2 つの Cloud Run サービスをもち、API エンドポイントによって接続先をルーティングします。

<img src="notebook-assets/1-1-goal.png" width=70%>

## 1.2. 変数・初期値の設定

本セクションで利用する環境変数・初期値を設定します。

In [ ]:
project_number="$(gcloud projects describe ${PROJECT_ID} --format='value(projectNumber)')"
echo "Project number:  ${project_number}"

cat << EOF >> /home/jupyter/config/.env
export api_name="my-apis-${RANDOM_KEY}"
export gateway_name="prod-gateway"
export service_account="${project_number}-compute@developer.gserviceaccount.com"
EOF

source /home/jupyter/config/.env
echo "Service account: ${service_account}"
echo "API name:        ${api_name}"

応答例）
```text
Project number:  12345678901
Service Account: 12345678901-compute@developer.gserviceaccount.com
API name:        my-apis-32fa0783-c5fd-4032-bebf-f2cc71a0cb3d
```

In [ ]:
gcloud config set compute/region asia-northeast1
gcloud config set compute/zone asia-northeast1-a
gcloud config set run/region asia-northeast1
gcloud config set run/platform managed

## 1.3. API の有効化

利用するサービス群を有効化します。

In [ ]:
echo "Enable Cloud Run API"
gcloud services enable run.googleapis.com
echo "Enable API Gateway API"
gcloud services enable apigateway.googleapis.com
echo "Enable Service Management API"
gcloud services enable servicemanagement.googleapis.com
echo "Enable Service Control API"
gcloud services enable servicecontrol.googleapis.com

## 1.4. API Gateway の作成

API Gateway のバックエンドとして Cloud Run を利用します。  

<img src="notebook-assets/1-4-run.png" width=70%>

コンテナに渡された環境変数や [クラウドのメタデータサーバ](https://cloud.google.com/compute/docs/storing-retrieving-metadata?hl=ja) にアクセスした結果を返す [REST API](https://github.com/pottava/http-return-everything) を Cloud Run にデプロイします。**エンドポイントは認証されたユーザーからのみアクセスを受け付ける、デフォルトの設定で作成**します。もし誰からでも接続を受け付けたい場合は `--allow-unauthenticated` オプションを付与してください。

In [ ]:
gcloud run deploy backend-apis --image gcr.io/pottava/re:v2.0 \
    --set-env-vars ENABLE_GCP=1,ENABLE_AWS=0 --quiet

応答例）
```text
..
Done.                                                                          
Service [backend-apis] revision [backend-apis-00001-xxx] has been deployed and is serving 100 percent of traffic.
Service URL: https://backend-apis-xxxxxx-an.a.run.app
```

デプロイされた [Cloud Run サービスのエンドポイントをバックエンドとした API の定義](resources/v1.yaml) を作成し、API Gateway の構成を v1 として作成します。まずは設定ファイルの差分を確認してみましょう。

<img src="notebook-assets/1-4-config.png" width=70%>

In [ ]:
api_endpoint="$(gcloud run services describe backend-apis --format 'value(status.url)')"
echo -e "Cloud Run endpoint: ${api_endpoint}\n"

find resources -type f -name "*.yaml" | xargs sed -ie "s|<backend-apis>|${api_endpoint}|g"
cat resources/v1.yaml | grep -B 2 -A 3 x-google

応答例）
```text
Cloud Run endpoint: https://backend-apis-xxxxxx-an.a.run.app

      ..
      x-google-backend:
        address: https://backend-apis-xxxxxx-an.a.run.app
      ..
```

ではこの設定で API を作成します。API そのものと設定をそれぞれ作成するため、完了まで **5 分前後かかります**。

In [ ]:
gcloud api-gateway api-configs create v1 \
    --api="${api_name}" --openapi-spec=resources/v1.yaml \
    --backend-auth-service-account="${service_account}"

応答例）
```text
Waiting for API [my-apis] to be created...done.
Waiting for API Config [v1] to be created for API [my-apis]...done. 
```

作成した v1 の構成で、実際に外部からの接続を受け付けるゲートウェイを作成します。5 分ほど待って応答が返ってきたら、そのエンドポイントにアクセスし、REST API から応答があることを確認してみましょう。

<img src="notebook-assets/1-4-gateway.png" width=70%>

In [ ]:
gcloud api-gateway gateways create "${gateway_name}" --location=asia-east1 \
    --api="${api_name}" --api-config=v1
echo "Endpoint: https://$( gcloud api-gateway gateways describe ${gateway_name} \
    --location=asia-east1 --format 'value(defaultHostname)' )"

応答例）
```text
Waiting for API Gateway [prod-gateway] to be created with [projects/your-project-id/locations/global/apis/my-apis/configs/v1] config...done.                    
Endpoint: https://prod-gateway-xxxxxxx.de.gateway.dev 
```

In [ ]:
curl -s "https://$( gcloud api-gateway gateways describe ${gateway_name} \
    --location=asia-east1 --format 'value(defaultHostname)' )" | jq -r '.googlecloud.gce.zone'

応答例）
```text
projects/12345678901/zones/asia-northeast1-1
```

## 1.5. Gateway aggregation pattern への更新

パスベース ルーティングにより、異なる API サービスをまとめることができることを見ていきましょう。  
ここでは、API Gateway のバックエンドとして 2 つの Cloud Run サービスを設定します。  
まずは 2 つ目の Cloud Run をデプロイします。

<img src="notebook-assets/1-5-run.png" width=70%>

In [ ]:
gcloud run deploy backend-errs --image gcr.io/pottava/errs:v1.1 --quiet

応答例）
```text
..
Done.                                                                          
Service [backend-errs] revision [backend-errs-00001-xxx] has been deployed and is serving 100 percent of traffic.
Service URL: https://backend-errs-xxxxxx-an.a.run.app
```

[2 つの REST API を集約した 1 つの API 定義](resources/v2.yaml) を作成し、それを構成 v2 として登録します。

<img src="notebook-assets/1-5-config.png" width=70%>

In [ ]:
err_endpoint="$(gcloud run services describe backend-errs --format 'value(status.url)')"
echo -e "Cloud Run endpoint: ${err_endpoint}\n"

find resources -type f -name "*.yaml" | xargs sed -ie "s|<backend-errs>|${err_endpoint}|g"
cat resources/v2.yaml | grep -B 1 -A 3 backend-errs

応答例）
```text
Backend: https://backend-errs-xxxxxx-an.a.run.app

      x-google-backend:
        address: https://backend-errs-xxxxxx-an.a.run.app/errors
        path_translation: APPEND_PATH_TO_ADDRESS
      ..
```
API Gateway では OpenAPI で定義したパラメタをバックエンドに受け渡す方法が [2 つ用意されています](https://cloud.google.com/api-gateway/docs/passing-data#generating_the_backend_service_url_from_an_api_request)。ここでは `APPEND_PATH_TO_ADDRESS` を指定することで、パラメタを URI パスに連結する形に変換します。

In [ ]:
gcloud api-gateway api-configs create v2 \
    --api="${api_name}" --openapi-spec=resources/v2.yaml \
    --backend-auth-service-account="${service_account}"

応答例）
```text
Waiting for API Config [v2] to be created for API [my-apis]...done. 
```

新しく作成した v2 の構成をゲートウェイに適用し、その挙動を変えてみます。

<img src="notebook-assets/1-1-goal.png" width=70%>

In [ ]:
gcloud api-gateway gateways update "${gateway_name}" --location=asia-east1 \
    --api="${api_name}" --api-config=v2

応答例）
```text
Waiting for API Gateway [prod-gateway] to be updated...done.
```

応答が変わったことを確認してみましょう。/errors/ 以下の API も同時に使えるようになっています。

In [ ]:
curl -i -XGET "https://$( gcloud api-gateway gateways describe ${gateway_name} \
    --location=asia-east1 --format 'value(defaultHostname)' )/errors/403"

応答例）
```text
HTTP/2 403
content-type: text/plain; charset=UTF-8
..

403 Forbidden
```

[次へ: 2-secure-apis.ipynb](2-secure-apis.ipynb)